In [19]:
#Computação científica
import numpy as np

#análise de dados
import pandas as pd

#machine learning

import sklearn

# from feature-engine
from feature_engine.imputation import (
    AddMissingIndicator,
    MeanMedianImputer,
    CategoricalImputer,
)

from feature_engine.transformation import YeoJohnsonTransformer

In [20]:
sklearn.set_config(display='diagram')
sklearn.set_config(transform_output="pandas")

# Introdução

## Motivação


## Objetivos


# Download Dataset

In [21]:
# O dataset está disponível no seguinte URL:
#https://www.kaggle.com/competitions/porto-seguro-safe-driver-prediction/data

path='/home/rodolfo/Insync/rodolfopcruz2@gmail.com/Google Drive/Estudo/Python_Projects/datasets/porto-seguro-safe-driver-prediction/'
train=pd.read_csv(path+'train.csv')
test= pd.read_csv(path+'test.csv')

# Feature Engineering

- Missing Values
- Distribuição não normal
- Remoção de labels raras nas variáveis categóricas
- Encoding
- Normalização

In [22]:
cat_features=[feature for feature in train.columns if 'cat' in feature]
bin_features=[feature for feature in train.columns if 'bin' in feature]
num_features=[feature for feature in train.columns if 'cat' not in feature 
                                                    and 'bin' not in feature and
                                                    feature!='target' and feature!='id']


## Missing Values

In [23]:
#Missing values estão com valor -1
#Substituir -1 por nan para facilitar a identificação dos valores ausentes

train=train.replace(-1, np.nan)
test =test.replace(-1,np.nan)

In [24]:
#Separar os dados de input dos dados de output

y_train=train['target'] #Resposta esperada (cliente solicitou ou não um seguro)
train=train.drop(columns='target')

### Features Binárias

- Nas colunas com baixa proporção de missing values eles serão substituídos pelo valor mais comum;
- Nas colunas com elevada proporção de missing values eles serão substituídos por uma string para identificar o valor ausente;


In [25]:
train.loc[:,bin_features].isna().sum()

ps_ind_06_bin     0
ps_ind_07_bin     0
ps_ind_08_bin     0
ps_ind_09_bin     0
ps_ind_10_bin     0
ps_ind_11_bin     0
ps_ind_12_bin     0
ps_ind_13_bin     0
ps_ind_16_bin     0
ps_ind_17_bin     0
ps_ind_18_bin     0
ps_calc_15_bin    0
ps_calc_16_bin    0
ps_calc_17_bin    0
ps_calc_18_bin    0
ps_calc_19_bin    0
ps_calc_20_bin    0
dtype: int64

In [26]:
np.shape(test)

(892816, 58)

In [27]:
#Os missing values das features binárias serão substituídos pelo valor mais frequente na coluna

train[bin_features]=train[bin_features].astype('object') #converter para formato categórico para permitir a substituição
test[bin_features] =test[bin_features].astype('object')   #converter para formato categórico para permitir a substituição

bin_imputer_most_frequent=CategoricalImputer(imputation_method='frequent',variables=bin_features)
bin_imputer_most_frequent.fit(train) #fit nos dados de treino

train=bin_imputer_most_frequent.transform(train) #transformnar dados de treino e de teste
test =bin_imputer_most_frequent.transform(test)

train[bin_features].isna().mean()

ps_ind_06_bin     0.0
ps_ind_07_bin     0.0
ps_ind_08_bin     0.0
ps_ind_09_bin     0.0
ps_ind_10_bin     0.0
ps_ind_11_bin     0.0
ps_ind_12_bin     0.0
ps_ind_13_bin     0.0
ps_ind_16_bin     0.0
ps_ind_17_bin     0.0
ps_ind_18_bin     0.0
ps_calc_15_bin    0.0
ps_calc_16_bin    0.0
ps_calc_17_bin    0.0
ps_calc_18_bin    0.0
ps_calc_19_bin    0.0
ps_calc_20_bin    0.0
dtype: float64

In [28]:
#Parametros que serão usados pelo imputer
bin_imputer_most_frequent.imputer_dict_

{'ps_ind_06_bin': 0,
 'ps_ind_07_bin': 0,
 'ps_ind_08_bin': 0,
 'ps_ind_09_bin': 0,
 'ps_ind_10_bin': 0,
 'ps_ind_11_bin': 0,
 'ps_ind_12_bin': 0,
 'ps_ind_13_bin': 0,
 'ps_ind_16_bin': 1,
 'ps_ind_17_bin': 0,
 'ps_ind_18_bin': 0,
 'ps_calc_15_bin': 0,
 'ps_calc_16_bin': 1,
 'ps_calc_17_bin': 1,
 'ps_calc_18_bin': 0,
 'ps_calc_19_bin': 0,
 'ps_calc_20_bin': 0}

### Features Categóricas

Duas estratégias para substituição doos missing values:

1) Features com proporção de ausentes inferior ao threshold estipulado:

        Substituição por valor mais comum

2) Features com proporção de ausentes superior ao threshold estipulado:
        
        Substituição dos valores ausentes pela string missing

In [29]:
cat_features

['ps_ind_02_cat',
 'ps_ind_04_cat',
 'ps_ind_05_cat',
 'ps_car_01_cat',
 'ps_car_02_cat',
 'ps_car_03_cat',
 'ps_car_04_cat',
 'ps_car_05_cat',
 'ps_car_06_cat',
 'ps_car_07_cat',
 'ps_car_08_cat',
 'ps_car_09_cat',
 'ps_car_10_cat',
 'ps_car_11_cat']

In [30]:
#proporção de missing values
train[cat_features].isna().mean().sort_values(ascending=False)


ps_car_03_cat    0.690898
ps_car_05_cat    0.447825
ps_car_07_cat    0.019302
ps_ind_05_cat    0.009760
ps_car_09_cat    0.000956
ps_ind_02_cat    0.000363
ps_car_01_cat    0.000180
ps_ind_04_cat    0.000139
ps_car_02_cat    0.000008
ps_car_04_cat    0.000000
ps_car_06_cat    0.000000
ps_car_08_cat    0.000000
ps_car_10_cat    0.000000
ps_car_11_cat    0.000000
dtype: float64

In [31]:
#necessário converter as variáveis categŕicas para objetos para permitir imput com valor mais frequente

train[cat_features] = train[cat_features].astype('object')
test [cat_features] = test[cat_features].astype('object')

In [32]:
missing_threshold=1/100
#features categóricas com número de missing values superior ao thresold
cat_features_muitos_na=train[cat_features].isna().mean()>missing_threshold
cat_features_muitos_na=cat_features_muitos_na[cat_features_muitos_na].index.to_list()

#features categóricas com número de missing values inferior ao thresold
cat_features_poucos_na=train[cat_features].isna().mean()<missing_threshold
cat_features_poucos_na=cat_features_poucos_na[cat_features_poucos_na].index.to_list()

In [33]:
#imputer para features categóricas com elevada proporção de missingf values
cat_imputer_muitos_na=CategoricalImputer(imputation_method='missing',fill_value='missing',variables=cat_features_muitos_na)
cat_imputer_muitos_na.fit(train)

CategoricalImputer(fill_value='missing',
                   variables=['ps_car_03_cat', 'ps_car_05_cat',
                              'ps_car_07_cat'])

In [34]:
#imputer para features categóricas com baixa proporção de missing values
cat_imputer_poucos_na=CategoricalImputer(imputation_method='frequent',variables=cat_features_poucos_na)
cat_imputer_poucos_na.fit(train)

CategoricalImputer(imputation_method='frequent',
                   variables=['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat',
                              'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_04_cat',
                              'ps_car_06_cat', 'ps_car_08_cat', 'ps_car_09_cat',
                              'ps_car_10_cat', 'ps_car_11_cat'])

In [35]:
train=cat_imputer_muitos_na.transform(train) #transformar os dados de treino
train=cat_imputer_poucos_na.transform(train)
test =cat_imputer_muitos_na.transform(test) #transformar os dados de teste
test =cat_imputer_poucos_na.transform(test)


train[cat_features].isna().mean()

ps_ind_02_cat    0.0
ps_ind_04_cat    0.0
ps_ind_05_cat    0.0
ps_car_01_cat    0.0
ps_car_02_cat    0.0
ps_car_03_cat    0.0
ps_car_04_cat    0.0
ps_car_05_cat    0.0
ps_car_06_cat    0.0
ps_car_07_cat    0.0
ps_car_08_cat    0.0
ps_car_09_cat    0.0
ps_car_10_cat    0.0
ps_car_11_cat    0.0
dtype: float64

In [36]:
#Parâmetros que serão usados pelo imputer

cat_imputer_muitos_na.imputer_dict_

{'ps_car_03_cat': 'missing',
 'ps_car_05_cat': 'missing',
 'ps_car_07_cat': 'missing'}

In [37]:
#Parâmetros que serão usados pelo imputer

cat_imputer_poucos_na.imputer_dict_

{'ps_ind_02_cat': 1.0,
 'ps_ind_04_cat': 0.0,
 'ps_ind_05_cat': 0.0,
 'ps_car_01_cat': 11.0,
 'ps_car_02_cat': 1.0,
 'ps_car_04_cat': 0,
 'ps_car_06_cat': 11,
 'ps_car_08_cat': 1,
 'ps_car_09_cat': 2.0,
 'ps_car_10_cat': 1,
 'ps_car_11_cat': 104}

### Features Numéricas

In [38]:
#proporção de missing values nas features numéricas
train[num_features].isna().mean()

ps_ind_01     0.000000
ps_ind_03     0.000000
ps_ind_14     0.000000
ps_ind_15     0.000000
ps_reg_01     0.000000
ps_reg_02     0.000000
ps_reg_03     0.181065
ps_car_11     0.000008
ps_car_12     0.000002
ps_car_13     0.000000
ps_car_14     0.071605
ps_car_15     0.000000
ps_calc_01    0.000000
ps_calc_02    0.000000
ps_calc_03    0.000000
ps_calc_04    0.000000
ps_calc_05    0.000000
ps_calc_06    0.000000
ps_calc_07    0.000000
ps_calc_08    0.000000
ps_calc_09    0.000000
ps_calc_10    0.000000
ps_calc_11    0.000000
ps_calc_12    0.000000
ps_calc_13    0.000000
ps_calc_14    0.000000
dtype: float64

In [39]:
missing_threshold=1/100

#numeric features com muitos mussing values
num_features_muitos_na=train[num_features].isna().mean()>missing_threshold
num_features_muitos_na=num_features_muitos_na[num_features_muitos_na].index.to_list()

#numeric features com poucos missing values
num_features_poucos_na=train[num_features].isna().mean()<missing_threshold
num_features_poucos_na=num_features_poucos_na[num_features_poucos_na].index.to_list()

In [40]:
#imputer com muitos na


missing_indicator=AddMissingIndicator(variables=num_features_muitos_na)
missing_indicator.fit(train) #fit nos dados de treino



AddMissingIndicator(variables=['ps_reg_03', 'ps_car_14'])

In [41]:
train=missing_indicator.transform(train) #transformar dados de treino e de teste
test =missing_indicator.transform(test)

In [42]:
#Todas os missing values nas features numéricas serão substituídas pela média
num_imputer=MeanMedianImputer(imputation_method='mean',variables=num_features)
num_imputer.fit(train) #fit nos dados de treino

MeanMedianImputer(imputation_method='mean',
                  variables=['ps_ind_01', 'ps_ind_03', 'ps_ind_14', 'ps_ind_15',
                             'ps_reg_01', 'ps_reg_02', 'ps_reg_03', 'ps_car_11',
                             'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15',
                             'ps_calc_01', 'ps_calc_02', 'ps_calc_03',
                             'ps_calc_04', 'ps_calc_05', 'ps_calc_06',
                             'ps_calc_07', 'ps_calc_08', 'ps_calc_09',
                             'ps_calc_10', 'ps_calc_11', 'ps_calc_12',
                             'ps_calc_13', 'ps_calc_14'])

In [43]:
# Parâmetros que serão usados pelo imputer
num_imputer.imputer_dict_

{'ps_ind_01': 1.9003783525869775,
 'ps_ind_03': 4.423318078264551,
 'ps_ind_14': 0.012451025852973394,
 'ps_ind_15': 7.2999217085677035,
 'ps_reg_01': 0.6109913778620055,
 'ps_reg_02': 0.4391843578422479,
 'ps_reg_03': 0.8940473268722899,
 'ps_car_11': 2.3460997602514757,
 'ps_car_12': 0.37994713350764386,
 'ps_car_13': 0.8132646756363582,
 'ps_car_14': 0.3746906387381256,
 'ps_car_15': 3.065899443364819,
 'ps_calc_01': 0.4497563893201079,
 'ps_calc_02': 0.44958922199149215,
 'ps_calc_03': 0.44984879337110145,
 'ps_calc_04': 2.3720808720254296,
 'ps_calc_05': 1.8858860372438728,
 'ps_calc_06': 7.689445105273415,
 'ps_calc_07': 3.005823135286251,
 'ps_calc_08': 9.22590438364818,
 'ps_calc_09': 2.339033823242811,
 'ps_calc_10': 8.43359004858773,
 'ps_calc_11': 5.441382230196972,
 'ps_calc_12': 1.4419181736927347,
 'ps_calc_13': 2.8722875210849246,
 'ps_calc_14': 7.539026430918732}

In [44]:
train=num_imputer.transform(train) #transformar dados de treino e de teste
test =num_imputer.transform(test)

train[num_features].isna().mean()

ps_ind_01     0.0
ps_ind_03     0.0
ps_ind_14     0.0
ps_ind_15     0.0
ps_reg_01     0.0
ps_reg_02     0.0
ps_reg_03     0.0
ps_car_11     0.0
ps_car_12     0.0
ps_car_13     0.0
ps_car_14     0.0
ps_car_15     0.0
ps_calc_01    0.0
ps_calc_02    0.0
ps_calc_03    0.0
ps_calc_04    0.0
ps_calc_05    0.0
ps_calc_06    0.0
ps_calc_07    0.0
ps_calc_08    0.0
ps_calc_09    0.0
ps_calc_10    0.0
ps_calc_11    0.0
ps_calc_12    0.0
ps_calc_13    0.0
ps_calc_14    0.0
dtype: float64

## Transformações

- Aplicação da transformação de Yeo-Jhoson as seguintes variáveis:

    - ps_reg_03
    - ps_car_12
    - ps_car_13
    - ps_car_14
    - ps_car_15
    - ps_reg_02


In [45]:
continuous_numerical_variables=['ps_reg_03','ps_car_12','ps_car_13','ps_car_14','ps_car_15','ps_reg_02'] #features foram selecionadas durante análise exploratória
yeo_transformer=YeoJohnsonTransformer(variables=continuous_numerical_variables)

In [46]:
train=yeo_transformer.fit_transform(train) #fit nos dados de treino
test=yeo_transformer.transform(test)

In [47]:
#Parâmetros que serão usado pleo transformer
yeo_transformer.lambda_dict_

{'ps_reg_03': np.float64(-1.5645071507026156),
 'ps_car_12': np.float64(-5.9072831878455165),
 'ps_car_13': np.float64(-2.5269376303019384),
 'ps_car_14': np.float64(-3.162784028225431)}

## Labels Raras